## BA4G

**Implement LeaderboardCyclopeptideSequencing**

Given: An integer N and a collection of integers Spectrum.

Return: LeaderPeptide after running LeaderboardCyclopeptideSequencing(Spectrum, N).

Link: https://rosalind.info/problems/ba4g/

In [ ]:
def get_mass():
    mass = {
        "G": 57,
        "A": 71,
        "S": 87,
        "P": 97,
        "V": 99,
        "T": 101,
        "C": 103,
        "I": 113,
        "L": 113,
        "N": 114,
        "D": 115,
        "K": 128,
        "Q": 128,
        "E": 129,
        "M": 131,
        "H": 137,
        "F": 147,
        "R": 156,
        "Y": 163,
        "W": 186,
    }

    return mass

Funkcije iz prethodnog zadatka koja se koristi

In [ ]:
def Expand(peptides):
  
  mass = get_mass()
  mass_list = np.unique(list(mass.values()))
  new_list = []

  for peptide in peptides:
    for mass in mass_list:
      x = peptide.copy()
      x.append(mass)
      new_list.append(x)

  return new_list

In [ ]:
# small changes - razlika je jer ovdje primamo brojeve a tamo smo primali stringove
def cyclospectrum(peptide):

    n = len(peptide)
    mass = get_mass()
    cyclospectrum = []

    cycle = peptide + peptide[:-1]
    
    cyclospectrum.append(0)
    
    # razlicito u odnosu na BA4C
    cyclospectrum.append(sum(peptide))

    for i in range(n):
      for j in range(1, n):
        subpeptide = cycle[i : i + j]
        # razlicito u odnosu na BA4C
        cyclospectrum.append(sum(subpeptide))

    return sorted( cyclospectrum)

In [ ]:
def CyclicPeptideScoring(peptide,experimental_spectrum):

  theoretical_spectrum = cyclospectrum(peptide)
  score = 0

  for item in theoretical_spectrum:
    if item in experimental_spectrum:
      score += 1
      experimental_spectrum.remove(item)

  return score

In [ ]:
def Trim(leaderboard, linear_spectrum, N):

  linear_scores = {}
  leaderboard_new = []

  for i,peptide in enumerate(leaderboard):
    #samo promijenjeno u Cyclic u  odnosu na BA4L
    linear_scores[i] = CyclicPeptideScoring(peptide,linear_spectrum)
 
  linear_scores_list = sorted(linear_scores.values(), reverse = True)
  
  #dodano
  if len(leaderboard) <= N:
   return leaderboard

  for index, score in linear_scores.items():
     if score >= linear_scores_list[N-1]:
       leaderboard_new.append(leaderboard[index])


  return leaderboard_new

In [ ]:
def Output_format(peptide):
  p = ''
  output = []
  for pep in peptide:
    p += str(pep) + '-'
  # :-1 da se makne - sa kraja, inace bi bilo 113-128-186-
  print(p[:-1])

Novo

In [ ]:
def LeaderboardCyclopeptideSequencing(N,spectrum):

  leaderboard = [[]]
  leaderPeptide = []
  leaderScore = 0

  while leaderboard:

    leaderboard = Expand(leaderboard)
    new_leaderboard = []
    parent_mass = spectrum[-1]

    for peptide in leaderboard:
      if sum(peptide) == parent_mass:
          if CyclicPeptideScoring(peptide, spectrum) >= leaderScore:
            leaderPeptide = peptide
            leaderScore = CyclicPeptideScoring(peptide, spectrum)
      if sum(peptide) <= parent_mass:
        new_leaderboard.append(peptide)

    leaderboard = Trim(new_leaderboard,spectrum, N)

  return leaderPeptide

In [ ]:
# dataset
with open('/content/rosalind_ba4g (3).txt') as file:
  N = file.readline().strip()
  spectrum = [int(s) for s in file.readline().strip().split()]

In [ ]:
result = LeaderboardCyclopeptideSequencing(int(N),spectrum)

In [ ]:
Output_format(result)

156-156-131-113-97-128-97-137-131-113-71-137-87-57-71
